# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [50]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [51]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [52]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith(('sk-proj-', 'sk-or-v1-')) and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")


API key looks good so far


In [53]:
# craft system prompt for AI technical asistance that explain technical questions concisely
system_prompt = """
 You are a technical tutor that assist learners in their technical learning. 
 Your primary objective is to provide precise, and easy to understand explanation to a given technical question.

Behavioural constraints
- Prefer correctness over verbosity.
- Ask clarification questions when requirements are underspecified.
- Do not invent missing facts.
- Explicitly state uncertainty.

Reasoning Approach:
- Analyze requirements before answering.
- Break complex problems into steps.
- State assumptions when needed.

Output Requirements:
- Use structured formatting.
- Provide concrete recommendations.
- Avoid speculation.
- Be concise but complete.

Interaction Guidelines:
- Ask clarification questions when ambiguity exists.
- Do not fabricate information.
- Maintain consistency across conversation turns.
"""

In [54]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openrouter = OpenAI(base_url="https://openrouter.ai/api/v1")
messages = [
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": question }
]

stream = openrouter.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)

full_output = ""
display_handle_ref = display(Markdown(""), display_id=True)
for chunk in stream:
    full_output += chunk.choices[0].delta.content or ""
    update_display(Markdown(full_output), display_id=display_handle_ref.display_id)


Certainly! Let's break down the code snippet you provided:

### Code Breakdown:
```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **Context**: 
   - This code is likely found within a generator function (as indicated by `yield from`). It allows the function to yield values to the caller.

2. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension:
     - It iterates over each `book` in the iterable `books`.
     - For each `book`, it attempts to access the value associated with the key `"author"` using `book.get("author")`.
     - The `if book.get("author")` clause filters out any entries where the author is `None` or doesn't exist (it ensures that only books with a valid author are considered).
     - The result is a set of authors, which automatically removes duplicates.

3. **Yielding**:
   - `yield from` allows the generator to yield each element from the set one by one. It effectively delegates yielding to the set comprehension, handing off control while also providing the values.

### What It Does:
- This code creates a set of unique authors from a list (or iterable) of books and yields them one by one. 
- It ignores any books that do not have an author defined.

### Why It Matters:
- **Efficiency**: Using a set deduplicates authors automatically, which can simplify the processing of unique elements.
- **Conciseness**: The use of a set comprehension combined with `yield from` keeps the code succinct while achieving its goal.

### Recommendation:
- Ensure that `books` is defined as an iterable (e.g., a list of dictionaries) where each dictionary represents a book with optional `"author"` keys.
  
If you need further clarification or have additional questions about this code's context, feel free to ask!

In [30]:
# download ollama model
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [49]:
# Get Llama 3.2 to answer
# ollama = OpenAI(base_url="http://localhost:11434/v1")
messages = [
    {"role": "system", "content": system_prompt },
    { "role": "user", "content": question}
]

ollama_stream = ollama.chat(model= MODEL_LLAMA, messages=messages, stream=True)

ollama_response = ""
ollama_display = display(Markdown(ollama_response), display_id=True)

for chunk in ollama_stream:
    message = chunk.get("message")
    if message:
        ollama_response += message.get("content", "")
        update_display(Markdown(ollama_response), display_id=ollama_display.display_id)



**Code Explanation**

This code uses a combination of Python's `yield` keyword, dictionary methods (`get()`), and generator expressions to extract authors' names from a list of book objects.

Here's a breakdown:

*   `{book.get("author") for book in books if book.get("author")}`: This is a generator expression that:
    *   Iterates over each `book` object in the `books` collection.
    *   Uses the `.get()` method to retrieve the value associated with the key `"author"` from each `book` dictionary. If the key does not exist, it returns `None`.
    *   Yields (or produces) each author's name that is successfully retrieved.

*   `yield from`: This keyword allows us to "forward" the yielded values from the inner generator expression (`{...}`) to an outer function or context. In this case, we use it in a `yield` statement within another function or context (not shown in your example).

**How It Works**

When `yield from {book.get("author") for book in books if book.get("author")}` is executed:

1.  The generator expression is evaluated.
2.  For each `book`, it tries to retrieve the author name using `.get("author")`.
3.  If the key exists, its value is yielded as a string.
4.  If the key does not exist, `None` is yielded.

The outer `yield from` statement then "forwards" these yielded values to any context that uses this expression.

**Example Use Case**

Suppose you have a collection of book objects and want to extract all their authors' names into a list:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"}
]

# Extracting authors using the given code
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

This will output: `Author A`, `Author B`. The third book has no author, so it is skipped.

**Advice**

If you want to handle cases where a book doesn't have an author (i.e., returns `None`), consider modifying the generator expression to skip such books or include them with a default value:

```python
# With None values
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)

# Including books without authors with a default name
for author in yield from {book.get("author", "Unknown Author") for book in books if book.get("author")}:
    print(author)
```